In [ ]:
!pip install python-dotenv
!pip install langchain_groq
!pip install langchain
!pip install openai-whisper
!pip install yt-dlp whisper
!pip install yt_dlp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 933.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  G

In [ ]:
import os
from dotenv import load_dotenv
from google.colab import userdata
from langchain_groq import ChatGroq
groq_api = userdata.get("GROQ_API")

# This is the YouTube video we're going to use.
YOUTUBE_VIDEO = "https://www.youtube.com/watch?v=cdiD-9MMpb0"
llm = ChatGroq(api_key=groq_api, model="llama3-8b-8192")

llm.invoke("hi")

AIMessage(content="Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 11, 'total_tokens': 36, 'completion_time': 0.020833333, 'prompt_time': 0.003278638, 'queue_time': 0.047453491, 'total_time': 0.024111971}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-a6d45378-2aa4-441a-9990-b8ffe37ed177-0', usage_metadata={'input_tokens': 11, 'output_tokens': 25, 'total_tokens': 36})

In [ ]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
chain=llm | parser
chain.invoke('hi')

"Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [ ]:
from langchain.prompts import ChatPromptTemplate
template = """
Answer the question based on the context below. If you can't
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
prompt.format(context="Mary's sister is Susana", question="Who is Mary's sister?")

'Human: \nAnswer the question based on the context below. If you can\'t\nanswer the question, reply "I don\'t know".\n\nContext: Mary\'s sister is Susana\n\nQuestion: Who is Mary\'s sister?\n'

In [ ]:
chain = prompt | llm | parser
chain.invoke({
    "context": "Mary's sister is Susana",
    "question": "Who is Mary's sister?"
})

'Susana'

In [ ]:
!pip install --user --upgrade yt-dlp
!pip install pytube
!pip install yt-dlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 981.1 kB/s eta 0:00:00


In [ ]:
import os
import whisper
import tempfile
import subprocess
from pytube import YouTube
import yt_dlp

In [ ]:
# URL of the YouTube video you want to download
youtube_url = "https://www.youtube.com/watch?v=cdiD-9MMpb0"
download_path = "./"  # Path where files will be saved

# Step 1: Download YouTube video and extract the audio as mp3
print("Downloading video and extracting audio...")
ydl_opts = {
    'format': 'bestaudio/best',
    'outtmpl': os.path.join(download_path, 'audio.%(ext)s'),
    'noplaylist': True,
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    info = ydl.extract_info(youtube_url, download=True)
    audio_file = ydl.prepare_filename(info)
    audio_file = audio_file.replace('.webm', '.mp3')  # Ensuring correct file extension
    print(f"Audio downloaded and saved to: {audio_file}")

# Step 2: Transcribe the audio using Whisper
print("Transcribing audio to text...")
whisper_model = whisper.load_model("base")  # You can use other models like 'small', 'medium', or 'large'

# Transcribing the audio file
transcription = whisper_model.transcribe(audio_file, fp16=False)["text"].strip()

# Step 3: Save the transcription to a text file
transcription_file = os.path.join(download_path, "transcription.txt")
with open(transcription_file, "w") as f:
    f.write(transcription)

print(f"Transcription saved to: {transcription_file}")


[youtube] Extracting URL: https://www.youtube.com/watch?v=cdiD-9MMpb0
[youtube] cdiD-9MMpb0: Downloading webpage
[youtube] cdiD-9MMpb0: Downloading ios player API JSON
[youtube] cdiD-9MMpb0: Downloading web creator player API JSON
[youtube] cdiD-9MMpb0: Downloading m3u8 information
[info] cdiD-9MMpb0: Downloading 1 format(s): 251
[download] ./audio.webm has already been downloaded
[download] 100% of  167.01MiB
[ExtractAudio] Destination: ./audio.mp3
Deleting original file ./audio.webm (pass -k to keep)
Audio downloaded and saved to: ./audio.mp3
Transcribing audio to text...


100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 113MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Transcription saved to: ./transcription.txt


In [ ]:
with open("transcription.txt") as file:
    transcription = file.read()

transcription[:100]

"I think it's possible that physics has exploits and we should be trying to find them, arranging some"

In [ ]:
transcription

"I think it's possible that physics has exploits and we should be trying to find them, arranging some kind of a crazy quantum mechanical system that somehow gives you buffer overflow, somehow gives you a rounding error in the floating point. Synthetic intelligences are kind of like the next stage of development. And I don't know where it leads to, like at some point, I suspect the universe is some kind of a puzzle. These synthetic AI's will uncover that puzzle and solve it. The following is a conversation with Andre Kappathi, previously the director of AI at Tesla. And before that, at OpenAI and Stanford, he is one of the greatest scientists, engineers, and educators in the history of artificial intelligence. This is the Lex Friedman podcast to support it. Please check out our sponsors and now, to your friends, here's Andre Kappathi. What is a neural network? And what does it seem to do such a surprisingly good job of learning? What is a neural network? It's a mathematical abstraction 

In [ ]:
try:
    chain.invoke({
        "context": transcription,
        "question": "Is reading papers a good idea?"
    })
except Exception as e:
    print(e)

Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01j7x0h4zcf8hrv1ez6k7st887` on tokens per minute (TPM): Limit 30000, Used 0, Requested 53670. Please try again in 47.34s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


In [ ]:
!pip install langchain-community
from langchain_community.document_loaders import TextLoader
loader= TextLoader('transcription.txt')
Text_documents=loader.load()
Text_documents[:50]

[Document(metadata={'source': 'transcription.txt'}, page_content="I think it's possible that physics has exploits and we should be trying to find them, arranging some kind of a crazy quantum mechanical system that somehow gives you buffer overflow, somehow gives you a rounding error in the floating point. Synthetic intelligences are kind of like the next stage of development. And I don't know where it leads to, like at some point, I suspect the universe is some kind of a puzzle. These synthetic AI's will uncover that puzzle and solve it. The following is a conversation with Andre Kappathi, previously the director of AI at Tesla. And before that, at OpenAI and Stanford, he is one of the greatest scientists, engineers, and educators in the history of artificial intelligence. This is the Lex Friedman podcast to support it. Please check out our sponsors and now, to your friends, here's Andre Kappathi. What is a neural network? And what does it seem to do such a surprisingly good job of lea

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter= RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=20)
documents=text_splitter.split_documents(Text_documents)

In [ ]:
%pip install -qU langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 842.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 14.2 MB/s eta 0:00:00


In [ ]:

%pip install langchain-google-genai

In [ ]:
import getpass
import os

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

vector = embeddings.embed_query("Hello world")
vector[:5]

[0.04703257977962494,
 -0.04019005596637726,
 -0.02902696281671524,
 -0.02680964209139347,
 0.01892058178782463]

In [ ]:
len(vector)

768

In [ ]:
sentence1 = embeddings.embed_query("Mary's sister is Susana")
sentence2 = embeddings.embed_query("Pedro's mother is a teacher")

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

query_sentence1_similarity = cosine_similarity([vector], [sentence1])[0][0]
query_sentence2_similarity = cosine_similarity([vector], [sentence2])[0][0]

query_sentence1_similarity, query_sentence2_similarity

(0.5763852912857031, 0.5650509037410656)

In [ ]:
!pip install docarray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.2/270.2 kB 2.7 MB/s eta 0:00:00


In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore1 = DocArrayInMemorySearch.from_texts(
    [
        "Mary's sister is Susana",
        "John and Tommy are brothers",
        "Patricia likes white cars",
        "Pedro's mother is a teacher",
        "Lucia drives an Audi",
        "Mary has two siblings",
    ],
    embedding=embeddings,
)

/usr/local/lib/python3.10/dist-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [ ]:
vectorstore1.similarity_search_with_score(query="Mary's sister is Susana",k=3)

[(Document(metadata={}, page_content="Mary's sister is Susana"),
  0.8252358357850126),
 (Document(metadata={}, page_content='Mary has two siblings'),
  0.7350097842464195),
 (Document(metadata={}, page_content='John and Tommy are brothers'),
  0.5907043843537617)]

In [ ]:
retriever1 = vectorstore1.as_retriever()
retriever1.invoke("Who is Mary's sister?")

[Document(metadata={}, page_content="Mary's sister is Susana"),
 Document(metadata={}, page_content='Mary has two siblings'),
 Document(metadata={}, page_content='John and Tommy are brothers'),
 Document(metadata={}, page_content='Patricia likes white cars')]

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

setup = RunnableParallel(context=retriever1, question=RunnablePassthrough())
setup.invoke("What color is Patricia's car?")

{'context': [Document(metadata={}, page_content='Patricia likes white cars'),
  Document(metadata={}, page_content='Lucia drives an Audi'),
  Document(metadata={}, page_content="Pedro's mother is a teacher"),
  Document(metadata={}, page_content="Mary's sister is Susana")],
 'question': "What color is Patricia's car?"}

In [ ]:
chain = setup | prompt | llm | parser
chain.invoke("What color is Patricia's car?")

'White'

In [ ]:
chain.invoke("What car does Lucia drive?")

'Audi'

In [ ]:
vectorstore2 = DocArrayInMemorySearch.from_documents(documents, embeddings)

In [ ]:
chain = (
    {"context": vectorstore2.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | llm
    | parser
)
chain.invoke("What is synthetic intelligence?")

'Based on the context, it appears that synthetic intelligence refers to the next stage of development in artificial intelligence, where machines are able to operate on a scale that is similar to the way humans operate. The speaker mentions that these synthetic AIs will uncover a puzzle and solve it, suggesting that they will be capable of complex problem-solving and potentially even surpassing human intelligence.'

In [ ]:
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')

In [ ]:
!pip install langchain_pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.10.5
    Uninstalling aiohttp-3.10.5:
      Successfully uninstalled aiohttp-3.10.5


In [ ]:
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')

In [ ]:

from langchain_pinecone import PineconeVectorStore

index_name = "video-rag2"
pinecone = PineconeVectorStore.from_documents(
    documents = documents,
    embedding = embeddings,
    index_name = index_name,

)

In [ ]:
pinecone.similarity_search_with_score("What is synthetic intelligence?")[:3]

[(Document(id='6ffbdf56-964a-4d93-a0cd-e10c1d44398e', metadata={'source': 'transcription.txt'}, page_content="I think it's possible that physics has exploits and we should be trying to find them, arranging some kind of a crazy quantum mechanical system that somehow gives you buffer overflow, somehow gives you a rounding error in the floating point. Synthetic intelligences are kind of like the next stage of development. And I don't know where it leads to, like at some point, I suspect the universe is some kind of a puzzle. These synthetic AI's will uncover that puzzle and solve it. The following is a conversation with Andre Kappathi, previously the director of AI at Tesla. And before that, at OpenAI and Stanford, he is one of the greatest scientists, engineers, and educators in the history of artificial intelligence. This is the Lex Friedman podcast to support it. Please check out our sponsors and now, to your friends, here's Andre Kappathi. What is a neural network? And what does it se

In [ ]:
chain = (
    {"context":pinecone.as_retriever(), "question":RunnablePassthrough()} | prompt | llm | parser
)

chain.invoke("What is Hollywood going to start doing?")

'According to the context, Hollywood will start using AI to generate scenes, which will completely open up new possibilities and potentially make it possible to make a movie like Avatar for under a million dollars.'

In [ ]:
chain.invoke("Who is the prime minister of India?")

"I don't know. The context provided is a collection of transcriptions of conversations about AI, AGI, and related topics, and does not mention the prime minister of India."